In [ ]:
from gurobipy import *
import numpy as np
import pandas as pd
import sys

## Load model

In [ ]:
model = read("./model/2.2.1.mps")
model.write("./model/2.2.1/model.lp")
print(f"Number of vars: {len(model.getVars())}")
print(f"Number of constraints: {len(model.getConstrs())}")
print(model.getVars())

## Read file

In [ ]:
f = pd.read_csv("./data/2.4/Facility.csv").iloc[:, 1:].to_numpy()
f

In [ ]:
t = pd.read_csv("./data/2.4/TransCost.csv", header=None).to_numpy()
t

In [ ]:
D = pd.read_csv("./data/2.4/DemandMinus.csv").iloc[:, 1:].to_numpy()
D

## Read vars

In [ ]:
x = {}
c = {}

for i in range(10):
    c[i] = model.getVarByName(f"chose[{i}]")
    for j in range(200):
        x[i, j] = model.getVarByName(f"x[{i},{j}]")

c

## Calculate 2.4

Calculate 2.2.1

In [ ]:
model.optimize()

Take the solution and plug back to model

In [ ]:
if model.status == GRB.Status.OPTIMAL:
    for i in range(10):
        if c[i].X:
            for j in range(200):
                if x[i, j].X != 0:
                    print(f"{i + 1} {j + 1} {x[i, j].X}")

Update the demand

In [ ]:
if model.status == GRB.Status.OPTIMAL:
    for i in range(10):
        if not c[i].X:
            model.addConstr(c[i] == 0)
    for j in range(200):
        temp = model.getConstrByName(f"shop-demand[{j}]")
        if temp: model.remove(temp)
        model.addConstr(quicksum(x[i, j] for i in range(10)) >= D[j, -1])

model.write("./model/2.4/model.lp")
model.optimize()

Solve the model again

In [ ]:
if model.status == GRB.Status.OPTIMAL:
    temp = sys.stdout
    sys.stdout = open("./out/2.4.out", "w")
    for i in range(10):
        print(round(c[i].X), end=" ")
    print()
    for i in range(10):
        if c[i].X:
            for j in range(200):
                if x[i, j].X != 0:
                    print(f"{i + 1} {j + 1} {x[i, j].X}")
    print(model.ObjVal)
    sys.stdout = temp